In [2]:
import twitter
import tweepy
from tweepy import API
from tweepy import Cursor
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
import time
from time import sleep
import os
import requests
from bs4 import BeautifulSoup

ck = os.environ.get('CK')
cs = os.environ.get('CS')
atk = os.environ.get('ATK')
ats = os.environ.get('ATS')

# print(os.environ.get(ck))

auth = tweepy.OAuthHandler(ck, cs)
auth.set_access_token(atk, ats)
auth_api = API(auth)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [3]:
# Get a LIST of friends that you are following
list_of_friends_id = api.friends_ids()
print(len(list_of_friends_id))
list_of_friends_id[:5]

56


[788524, 2768501, 14138785, 6017542, 15808647]

In [222]:
# To get the ids of people that are following you
x = api.followers_ids()
print(len(x))
x[:5]

38


[17116782,
 1129863867960635396,
 264924050,
 731732329572376576,
 994672306013261824]

In [39]:
# Getting informations from other users timelines
y = api.user_timeline(15754281,count=5)
print(type(y),len(y))

<class 'tweepy.models.ResultSet'> 5


In [43]:
print(y[0].geo,y[0].id,y[0].favorite_count,y[0].retweet_count,y[0].lang)

None 1134764021163409408 25 3 en


In [17]:
# my id
my_id = api.me().id
len(api.user_timeline(my_id))

20

In [44]:
# Getting my id and taking info from it
my_id = api.me().id
y = api.user_timeline(my_id)
my_tweets_list = [i.id for i in y]

In [4]:
list_of_friends_id = api.friends_ids()

def theupdate_at_work():
    while True:
        df = pd.DataFrame(columns=np.arange(11))
        for user in list_of_friends_id[:5]:
            list_tweet = api.user_timeline(user,count=3,exclude_replies=True)
            for tweet in list_tweet:
                try:
                    hashtags = ' '.join([i for i in tweet._json['text'].split() if i[0]=='#'])
                    url=tweet._json['entities']['urls'][0]['url']
                    page = requests.get(url)
                    soup = BeautifulSoup(page.text, 'html.parser')
                    h1 = '#news '+soup.find('h1').text.strip()

                    df.loc[len(df)] = [tweet.id, #0
                                       tweet.geo, #1
                                       tweet.favorite_count, #2
                                       tweet.retweet_count, #3
                                       tweet.lang, #4
                                        user,  #5
                                       tweet._json['user']['id'],  #6
                                       tweet._json['text'],  #7
                                       tweet._json['entities']['urls'][0]['url'],   #8
                                       hashtags,  #9
                                       h1+' '+url+' '+hashtags]        #10
                except:
                    pass
        df[11]=df[2]+df[3]*2   #scoring of the tweets
        df = df[df[4]=='en']
        df[12] = [False if i[:2]=='RT' else True for i in df[7]]
        df = df.sort_values(by=11,ascending=False)
        df = df[df[12]]
        counter = 0
        counter_tweet = 0
        counter_errors = 0
        for tw in list(df[10]):
            # counter+=1
            # print('Global counter: ',counter, end='\r')
            if counter_tweet==5:
                break
            else:
                try:
                    api.update_status(tw)
                    counter_tweet+=1
                    #print('tweet number: ',counter_tweet,end='\r')
                    for second in range(180):
                        time.sleep(1)
                        #print('timer: ',179-second,end='\r')
                except:
                    pass

In [ ]:
theupdate_at_work()

In [319]:
tweet = api.user_timeline(15754281,count=3,exclude_replies=True)

In [321]:
tweet[0]._json['entities']['urls'][0]

{'url': 'https://t.co/iFVAavWvPU',
 'expanded_url': 'https://twitter.com/i/web/status/1134823744076222466',
 'display_url': 'twitter.com/i/web/status/1…',
 'indices': [117, 140]}

In [240]:
tweet[0]._json['entities']['urls'][0]['url']

'https://t.co/Lf6AqowKyN'

In [241]:
import selenium

In [308]:
df.shape

(22, 11)

In [257]:
url='https://t.co/Lf6AqowKyN'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [265]:
'#news '+soup.find('h1').text.strip()

'#news MLB sets record for most home runs in a month'

In [269]:
api.update_status(('#news '+soup.find('h1').text.strip()+' '+url))

TweepError: [{'code': 187, 'message': 'Status is a duplicate.'}]

In [340]:
df = pd.DataFrame(columns=np.arange(11))
for user in list_of_friends_id[:5]:
    list_tweet = api.user_timeline(user,count=3,exclude_replies=True)
    for tweet in list_tweet:
        try:
            hashtags = ' '.join([i for i in tweet._json['text'].split() if i[0]=='#'])
            url=tweet._json['entities']['urls'][0]['url']
            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'html.parser')
            h1 = '#news '+soup.find('h1').text.strip()

            df.loc[len(df)] = [tweet.id, #0
                               tweet.geo, #1
                               tweet.favorite_count, #2
                               tweet.retweet_count, #3
                               tweet.lang, #4
                                user,  #5
                               tweet._json['user']['id'],  #6
                               tweet._json['text'],  #7
                               tweet._json['entities']['urls'][0]['url'],   #8
                               hashtags,  #9
                               h1+' '+url+' '+hashtags]        #10
        except:
            pass
print(df.shape)
df[11]=df[2]+df[3]*2   #scoring of the tweets
df = df[df[4]=='en']
df[12] = [False if i[:2]=='RT' else True for i in df[7]]
df = df.sort_values(by=11,ascending=False)
df = df[df[12]]

(14, 11)


In [343]:
df[10][0]

'#news Beware high-value euro notes, British holidaymakers told https://t.co/wijl7h5j1D '

In [ ]:
np.arange(10)

In [307]:
' '.join([i for i in df[7][23].split() if i[0]=='#'])

'#PrideMonth #stance'

In [299]:
df.head(2)

,0,1,2,3,4,5,6,7,8,9,10
23,1134813479397920769,None,226,74,en,14511951,14511951,Taylor Swift kicked off #PrideMonth by knockin...,https://t.co/2VORF18j4E,374,True
25,1134802957592092672,None,103,24,en,14511951,14511951,Anita Hill gave the address for the graduating...,https://t.co/IJT44NKrMM,151,True
